In [4]:
import strawberryfields as sf
from strawberryfields.ops import *
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import colors, colorbar, cm
import time
from tqdm.notebook import tqdm

In [12]:
def variational_quantum_circuit(params):
    s = time.perf_counter()
    # Gate layer: D-R-S-R-K
    def layer(i, q):
        with tf.name_scope('layer_{}'.format(i)):
            Rgate(params[i*7+0])                | q
            Sgate(params[i*7+1], params[i*7+2]) | q
            Rgate(params[i*7+3])                | q
            Dgate(params[i*7+4], params[i*7+5]) | q
            Kgate(params[i*7+6])                | q
        return q
    
    # Start SF program
    prog = sf.Program(1)

    # construct the circuit
    with prog.context as q:
        for k in range(depth):
            q = layer(k, q)
    
    eng = sf.Engine("tf", backend_options={"cutoff_dim": dim})
    state = eng.run(prog, run_options={"eval": False}).state

    e = time.perf_counter()  
    #print('time',e-s)
    return state, e-s

In [25]:
dim = 1000
depth = 8

t = np.zeros(10)
for i in tqdm(range(10)):
    params = tf.Variable(tf.random.normal(shape=[7*depth], stddev=0.1)) # 最初のパラメータ
    state, t[i] = variational_quantum_circuit(params)
print('avg',np.sum(t/10))
print(state)

  0%|          | 0/10 [00:00<?, ?it/s]

avg 6.495052410000063
<FockStateTF: num_modes=1, cutoff=1000, pure=True, batched=False, hbar=2, dtype=<dtype: 'complex64'>>


In [39]:
dim = 500
# 単純な行列の積
state1_real = np.random.rand(dim, dim)
state1_imag = np.random.rand(dim, dim)
state1_np = state1_real + 1j * state1_imag
state1_qobj = Qobj(state_np)

state2_real = np.random.rand(dim, 1)
state2_imag = np.random.rand(dim, 1)
state2_np = state2_real + 1j * state2_imag
state2_qobj = Qobj(state_np)

itr = 100

s_np = time.perf_counter()
for i in range(itr):
    a_np = state1_np @ state2_np
e_np = time.perf_counter()

s_qobj = time.perf_counter()
for i in range(itr):
     a_qobj = state1_qobj * state2_qobj
e_qobj = time.perf_counter()

print('time_np', e_np-s_np)
print('time_qobj', e_qobj-s_qobj)

time_np 0.0019277999999758322
time_qobj 20.696699999999964


In [40]:
dim = 500
# 指数行列の計算
state1_real = np.random.rand(dim, dim)
state1_imag = np.random.rand(dim, dim)
state1_np = state1_real + 1j * state1_imag
state1_qobj = Qobj(state_np)

itr = 100

s_qobj = time.perf_counter()
for i in range(itr):
    a_qobj = state1_qobj.expm()
e_qobj = time.perf_counter()

s_np = time.perf_counter()
for i in range(itr):
    a_np = expm(state1_np)
e_np = time.perf_counter()

print('time_np', e_np-s_np)
print('time_qobj', e_qobj-s_qobj)

time_np 20.8004569000002
time_qobj 21.33832369999982
